# Tiles in `wsidata`

Tiles in wsidata is stored as a `GeoDataFrame` with an associated `TileSpec` object.

To extract tiles from a non-existing magnification level, the tiling operation in `wsidata` is recorded with following information:

- User specifies the tile size and magnification level.
- The actual tile size and magnification level are used to create the tiles.
- The tile size at the level 0.

Noted that `wsidata` cannot create tiles, it can only record the information for generating tiles. Please use LazySlide to create tiles.

In [ ]:
import pooch

s = pooch.retrieve(
    "https://openslide.cs.cmu.edu/download/openslide-testdata/Aperio/CMU-1-Small-Region.svs",
    fname="CMU-1-Small-Region.svs",
    known_hash="sha256:ed92d5a9f2e86df67640d6f92ce3e231419ce127131697fbbce42ad5e002c8a7",
)

In [ ]:
from wsidata import TileSpec, open_wsi

wsi = open_wsi(s)

In [ ]:
wsi.properties

Here we have a wsi with only one level with mpp=0.5

If we want to request a tile size of 100x100 at level 0, the tile operation will be performed at level 0 with a tile size of 100x100.

In [ ]:
TileSpec.from_wsidata(wsi, 100)

However, if we request a tile size of 100x100 at mpp=1, but mpp=1 doesn't exist in the original image.
The tile operation will be performed at level 0 with a tile size of 200x200 and rescaled to 100x100.

In [ ]:
TileSpec.from_wsidata(wsi, 100, mpp=1)

Of course, tiles with overlapping is also supported

In [ ]:
TileSpec.from_wsidata(wsi, 100, stride_px=50)

In [ ]:
TileSpec.from_wsidata(wsi, 100, overlap=0.1)

If you want to create a new tile table manually, please remember to synchronize the `TileSpec` with it.


In [ ]:
import numpy as np

from wsidata.io import add_tiles, sync_tile_spec

add_tiles(
    wsi,
    "tiles",
    np.random.randint(0, 255, (100, 2), dtype=np.int32),
    tile_spec=TileSpec.from_wsidata(wsi, 25),
    tissue_ids=np.random.randint(0, 2, 100),
)

wsi["new_tiles"] = wsi["tiles"].sample(10)
sync_tile_spec(wsi, "tiles", "new_tiles")